In [1]:
import pandas as pd
from arrow import Arrow

df = pd.read_csv(filepath_or_buffer='/kaggle/input/tokyo-weatherdata/weather_tokyo_data.csv')
df['date'] = df.apply(axis=1, func=lambda x: Arrow(year=x['year'], month=int(x['day'].split('/')[0]),
                                          day=int(x['day'].split('/')[1])).date())
df.columns = [item.strip() for item in df.columns]
df.head()

,year,day,temperature,humidity,atmospheric pressure,date
0,2022,11/6,13.5,61.0,1019.3,2022-11-06
1,2022,11/7,13.7,70.0,1018.9,2022-11-07
2,2022,11/8,15.9,55.0,1016.1,2022-11-08
3,2022,11/9,14.6,58.0,1022.2,2022-11-09
4,2022,11/10,15.2,64.0,1020.1,2022-11-10


In [2]:
df.columns

Index(['year', 'day', 'temperature', 'humidity', 'atmospheric pressure',
       'date'],
      dtype='object')